In [1]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv
import wandb

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

df_original = pd.read_csv('data/combined_old.csv')

df_enhanced = pd.read_csv('data/combined_new.csv')

In [2]:
df_enhanced

,ID_Key,ID,Company,Year,Presence,Index
0,1215.0,105.0,ADLER Real Estate AG (2015),2002,NaN,NaN
1,1215.0,105.0,ADLER Real Estate AG (2015),2003,NaN,NaN
2,1215.0,105.0,ADLER Real Estate AG (2015),2004,NaN,NaN
3,1215.0,105.0,ADLER Real Estate AG (2015),2005,NaN,NaN
4,1215.0,105.0,ADLER Real Estate AG (2015),2006,NaN,NaN
...,...,...,...,...,...,...
3961,NaN,56.0,GAGFAH S.A.,2015,NaN,NaN
3962,NaN,140.0,Highlight Communications AG,2015,NaN,NaN
3963,NaN,208.0,QIAGEN N.V.,2015,64.30,TecDAX
3964,NaN,89.0,RTL Group S.A.,2015,82.93,MDAX


In [3]:
df_original

,ID_Key,ID,Company,Year,Presence,Index
0,1215.0,105.0,ADLER Real Estate AG (2015),2002,NaN,NaN
1,1215.0,105.0,ADLER Real Estate AG (2015),2003,NaN,NaN
2,1215.0,105.0,ADLER Real Estate AG (2015),2004,NaN,NaN
3,1215.0,105.0,ADLER Real Estate AG (2015),2005,NaN,NaN
4,1215.0,105.0,ADLER Real Estate AG (2015),2006,NaN,NaN
...,...,...,...,...,...,...
3961,NaN,56.0,GAGFAH S.A.,2015,NaN,NaN
3962,NaN,140.0,Highlight Communications AG,2015,NaN,NaN
3963,NaN,208.0,QIAGEN N.V.,2015,64.30,TecDAX
3964,NaN,89.0,RTL Group S.A.,2015,82.93,MDAX


In [4]:
# First, add suffixes to each column in both dataframes except for the index
df_original_suffixed = df_original.add_suffix('_original')
df_enhanced_suffixed = df_enhanced.add_suffix('_enhanced')

# Join both dataframes based on their index
merged_df = pd.concat([df_original_suffixed, df_enhanced_suffixed], axis=1)

# drop the ID_Key_enhanced, ID_enhanced, Company_enhanced, and Index_enhanced columns
merged_df = merged_df.drop(columns=['ID_Key_enhanced', 'ID_enhanced', 'Company_enhanced', 'Year_enhanced', 'Index_enhanced'])
merged_df

,ID_Key_original,ID_original,Company_original,Year_original,Presence_original,Index_original,Presence_enhanced
0,1215.0,105.0,ADLER Real Estate AG (2015),2002,NaN,NaN,NaN
1,1215.0,105.0,ADLER Real Estate AG (2015),2003,NaN,NaN,NaN
2,1215.0,105.0,ADLER Real Estate AG (2015),2004,NaN,NaN,NaN
3,1215.0,105.0,ADLER Real Estate AG (2015),2005,NaN,NaN,NaN
4,1215.0,105.0,ADLER Real Estate AG (2015),2006,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3961,NaN,56.0,GAGFAH S.A.,2015,NaN,NaN,NaN
3962,NaN,140.0,Highlight Communications AG,2015,NaN,NaN,NaN
3963,NaN,208.0,QIAGEN N.V.,2015,64.30,TecDAX,64.30
3964,NaN,89.0,RTL Group S.A.,2015,82.93,MDAX,82.93


## Creating a large test set

This one includes the rows that are have been provided by the Kleinanlegerschutzverbund (?)

## Creating the test set

Only the rows Linus has checked manually

In [5]:
# drop all rows where merged_df['Presence_enhanced'] isna. Effectively, I only need ID_Key_original and Presence_enhanced, but the other columns are useful for debugging
test_set_large = merged_df.dropna(subset=['Presence_enhanced'])
test_set_large.to_csv('data/231215_test_set_large.csv', index=False)

In [6]:
# Filter out the rows where 'Presence_original' is NaN and 'Presence_enhanced' is not NaN
test_set_small = merged_df[merged_df['Presence_original'].isna() & merged_df['Presence_enhanced'].notna()]

# The result_df will contain the desired rows with distinct column suffixes
test_set_small.to_csv('data/231215_test_set_small.csv', index=False)

## Finding the correct file

Finding the file path to the PDF the data has been extracted from

### calculating pricing

Creating a function to calculate the cost of all this, to be used later

In [7]:
def calculate_cost(input_tokens, output_tokens, input_price_per_1000=0.01, output_price_per_1000=0.03):
    # Calculate the cost for input tokens
    input_cost = (input_tokens / 1000) * input_price_per_1000

    # Calculate the cost for output tokens
    output_cost = (output_tokens / 1000) * output_price_per_1000

    # Total cost
    total_cost = input_cost + output_cost

    return total_cost

In [8]:
import pdfplumber
from tqdm import tqdm
import ast
import re

directory = "data/Praesenzen_hv-info"

# Initialize Weights & Biases
wandb.init(project="hv-praesenzen")

# Define W&B Table to store results
columns = ["ID_Key_original", "Year_original", "Presence_enhanced", "Presence_predicted", "correct", "error", "price", "file_path", "standard_deviation", "mean", "comment"]
table = wandb.Table(columns=columns)

for index, (df_index, row) in enumerate(tqdm(test_set_small.iterrows(), total=test_set_small.shape[0])):

    if index == 2:
        break

    error = False
    error_during_page_assessment = False
    alternative_document_structure = False

    # total cost pf processing the document
    cost_total = 0

    # standard deviation of the highest percentage of each document to the other values for the same document
    std_dev = 0

    # mean of the highest percentage of each document to the other values for the same document
    mean = 0

    # add a comment collection variable for the case some pages dont output a number
    comment_collection = ""

    prediction_correct = False

    id_value = str(int(row['ID_Key_original']))
    year_value = str(int(row['Year_original']))

    # Initialize variable to store the found directory path
    found_directory_path = None

    # Find subdirectory
    for subdirectory in os.listdir(directory):
        subdirectory_path = os.path.join(directory, subdirectory)
        if os.path.isdir(subdirectory_path) and subdirectory.endswith(id_value):
            found_directory_path = subdirectory_path
            break

    # add "ASM" subdirectory to the path
    if found_directory_path:
        found_directory_path = os.path.join(found_directory_path, "ASM")

    # If a matching subdirectory is found, search for the correct file
    if found_directory_path:
        for file in os.listdir(found_directory_path):
            file_path = os.path.join(found_directory_path, file)
            if os.path.isfile(file_path) and file.endswith(year_value[-2:] + ".pdf"):
                # Found the file, you can add your code here to handle the file
                print(f"Found file: {file_path}")
                break
    else:
        # Handle the case where no matching subdirectory is found
        print(f"No subdirectory found for ID {id_value}")

    with pdfplumber.open(file_path) as pdf:
        
        # List to store all texts
        full_text = ""
 
        # Iterate through each page
        for page_number, page in enumerate(pdf.pages, start=1):
            # Extract text from the page
            page_text = page.extract_text()
            if page_text == "":
                test_set_small.at[index, 'error'] = "document could not be read"
                error = True
                break
            # Store the text
            # all_texts.append((page_number, page_text))
            full_text += page_text

        if error:
            table.add_data(id_value, year_value, row['Presence_enhanced'], 0, None, error, cost_total, file_path, std_dev, mean, comment_collection)
            
            error = False
            print("There was an error. Not evaluating this document.")
            continue

    highest_percentage_list = []

    system_prompt = "Du bist ein hilfreicher Assistent, der Berichte von Hauptversammlungen auswertet."
    print(system_prompt)
    user_prompt = "Im folgenden erhältst du einen Bericht einer Hauptversammlung. Das Dokument enthält eine Tabelle mit einer Kopfzeile, aber die Kopfzeile ist beim Extrahieren des Texts beschädigt worden. Bitte gib mir die volle Bezeichnung jeder Spalte in der korrekten Reihenfolge, wie sie im Dokument auftaucht. Antworte ausschließlich mit einer Liste im Format [spalte_1, spalte_2, spalte_3]. Wenn du keine Kopfzeile finden kannst, antworte mit [0]. Bericht: "
    print(user_prompt)
    combined_prompt = user_prompt + full_text

    # Call the GPT-4 chat_completion model
    response = client.chat.completions.create(model="gpt-4-1106-preview",  # Specify the model, e.g., "gpt-4"
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": combined_prompt}
    ],
    temperature=0.2)

    # Regular expression pattern to check if string starts with '[' and ends with ']'
    pattern = r'^\[.*\]$'

    # Check if the response string matches the pattern
    if re.match(pattern, response.choices[0].message.content):
        # Process the string if it matches
        column_names =response.choices[0].message.content # Adjust as needed
        print("column names: " + column_names)

        # check if response is 0. In that case, we check if the file might not be structured like a table, but more like a list (alternative document structure)
        if column_names == "[0]":

            user_prompt = "Im folgenden erhältst du einen Bericht einer Hauptversammlung. Werden in dem Dokument wiederholt und ausdrücklich Angaben zum auf der Versammlung vertretenen Grundkapital in Prozent gemacht (Also bspw. 'Grundkapital: 30%'? Antworte nur mit [1] oder [0]. Bericht: "
            print(user_prompt)
            combined_prompt = user_prompt + full_text

            # Call the GPT-4 chat_completion model
            response = client.chat.completions.create(model="gpt-4-1106-preview",  # Specify the model, e.g., "gpt-4"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": combined_prompt}
            ],
            temperature=0.2)

            print(response.choices[0].message.content)

            # calculate cost
            input_tokens = response.usage.prompt_tokens
            output_tokens = response.usage.completion_tokens
            cost_alternative_doc_structure_detection = calculate_cost(input_tokens, output_tokens)

            cost_total += cost_alternative_doc_structure_detection

            # check response
            if response.choices[0].message.content == "[1]":

                alternative_document_structure = True

                # find the highest grundkapital percentage in the document
                user_prompt = "Im folgenden erhältst du einen Bericht von einer Hauptversammlung. Antworte ausschließlich mit einer Liste im Format [zahl_1, zahl_2, zahl_3], die ausschließlich alle die genannten Prozentzahlen enthält, die sich auf den Prozentsatz des auf der Hauptversammlung vertretenen Grundkapitals beziehen. Wenn du dir nicht absolut sicher bist, antworte mit [0]. Verwende Punkt statt Komma für die Zahlen. Bericht: "
                print(user_prompt)
                combined_prompt = user_prompt + full_text

                # Call the GPT-4 chat_completion model
                response = client.chat.completions.create(model="gpt-4-1106-preview",  # Specify the model, e.g., "gpt-4"
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": combined_prompt}
                ],
                temperature=0.2)

                print(response.choices[0].message.content)

                # calculate cost
                input_tokens = response.usage.prompt_tokens
                output_tokens = response.usage.completion_tokens
                cost_alternative_doc_structure_extraction = calculate_cost(input_tokens, output_tokens)

                cost_total += cost_alternative_doc_structure_extraction

            elif response.choices[0].message.content == "[0]":
                comment_collection += "Keine Kopfzeile gefunden"
                error = True
                table.add_data(id_value, year_value, row['Presence_enhanced'], 0, None, error, cost_total, file_path, std_dev, mean, comment_collection)
                error_during_page_assessment = True
                continue

            else:
                comment_collection += response.choices[0].message.content + "\n"
                error = True
                table.add_data(id_value, year_value, row['Presence_enhanced'], 0, None, error, cost_total, file_path, std_dev, mean, comment_collection)
                error_during_page_assessment = True
                continue

    else:
        # Handle the case where the string doesn't match
        print("String does not start with '[' and end with ']'")
        comment_collection += response.choices[0].message.content + "\n"
        error = True
        table.add_data(id_value, year_value, row['Presence_enhanced'], 0, None, error, cost_total, file_path, std_dev, mean, comment_collection)
        error_during_page_assessment = True
        break

    if alternative_document_structure == False:
        user_prompt = "Im folgenden erhältst du einen Bericht einer Hauptversammlung. Das Dokument enthält eine Tabelle mit einer Kopfzeile, aber die Kopfzeile ist beim Extrahieren des Texts beschädigt worden. Die korrekte Kopfzeile habe ich angehängt. Antworte ausschließlich mit einer Liste im Format [zahl_1, zahl_2, zahl_3], die ausschließlich alle die genannten Prozentzahlen enthält, die sich auf den Prozentsatz des auf der Hauptversammlung vertretenen Grundkapitals beziehen. Durchsuche das ganze Dokument nach solchen Zahlen, auch außerhalb von Tabellen. Wenn du dir nicht absolut sicher bist, antworte mit [0]. Verwende Punkt statt Komma für die Zahlen. \n Korrekte Kopfzeile: "
        print("user_prompt: ", user_prompt)
        combined_prompt = user_prompt + column_names + "\n Bericht der Hauptversammlung: " + full_text

        # Call the GPT-4 chat_completion model
        response = client.chat.completions.create(model="gpt-4-1106-preview",  # Specify the model, e.g., "gpt-4"
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": combined_prompt}
        ],
        temperature=0.2)

        # calculate cost
        input_tokens = response.usage.prompt_tokens
        output_tokens = response.usage.completion_tokens
        cost_header_detection = calculate_cost(input_tokens, output_tokens)

        cost_total += cost_header_detection

        # Print the response
        print(response.choices[0].message.content)

    # get highest value within page
    try:
        percentage_list = ast.literal_eval(response.choices[0].message.content)
        highest_percentage_of_page = max(percentage_list)
    except SyntaxError:
        highest_percentage_of_page = 0
        comment_collection += response.choices[0].message.content + "\n"
        error = True

    # get highest value of all pages
    try:
        # highest_percentage_list.append(percentage_intro)
        highest_percentage_list.append(highest_percentage_of_page)
        highest_percentage = max(highest_percentage_list)
        # round highest percentage to 2 decimal places
        highest_percentage = round(highest_percentage, 2)
    except ValueError:
        highest_percentage = 0
        comment_collection += response.choices[0].message.content + "\n"
        error = True


    if error == False:
        # Convert the percentages to a numpy array for statistical calculations
        percentages = np.array(highest_percentage_list)

        # Calculate mean and standard deviation
        mean = np.mean(percentages)
        std_dev = np.std(percentages)

        # check if highest_percentage is more than 15 away from the second highest percentage
        if len(highest_percentage_list) > 1:
            second_highest_percentage = sorted(highest_percentage_list, reverse=True)[1]
            if highest_percentage - second_highest_percentage > 10:
                comment_collection += "Der ermittelte Wert weicht um mehr als 10 Prozentpunkte vom zweithöchsten Wert ab."
                error = True

        if error:
            table.add_data(id_value, year_value, row['Presence_enhanced'], highest_percentage, None, error, cost_total, file_path, std_dev, mean, comment_collection)
            continue

    print("highest_percentage: ", highest_percentage)

    # calculate cost
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens
    cost_page_analysis = calculate_cost(input_tokens, output_tokens)

    cost_total += cost_page_analysis

    if error_during_page_assessment:
        continue

    test_set_small.at[index, 'Presence_predicted'] = highest_percentage

    if row['Presence_enhanced'] == highest_percentage:
        prediction_correct = True
    else:
        # Check for another row with the same ID_Key_original and Year_original
        same_id_year_rows = test_set_small[(test_set_small['ID_Key_original'] == row['ID_Key_original']) & 
                            (test_set_small['Year_original'] == row['Year_original'])]

        # Check if any of those rows have Presence_enhanced equal to highest_percentage
        if any(same_id_year_rows['Presence_enhanced'] == highest_percentage):
            prediction_correct = True
            comment_collection += "Der ermittelte Wert stammt aus dem anderen Bericht diesen Jahres und ist dort korrekt ermittelt."
        else:
            prediction_correct = False

    table.add_data(id_value, year_value, row['Presence_enhanced'], highest_percentage, prediction_correct, error, cost_total, file_path, std_dev, mean, comment_collection)

print("system_prompt: ", system_prompt)
print("user_prompt: ", user_prompt)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: felixringe (fuels). Use `wandb login --relogin` to force relogin


  0%|          | 0/182 [00:00<?, ?it/s]

Found file: data/Praesenzen_hv-info/Adler Real Estate AG-1215/ASM/HV-Beschluss zur ordentlichen Hauptversammlung am 27.08.10.pdf
Du bist ein hilfreicher Assistent, der Berichte von Hauptversammlungen auswertet.
Im folgenden erhältst du einen Bericht einer Hauptversammlung. Das Dokument enthält eine Tabelle mit einer Kopfzeile, aber die Kopfzeile ist beim Extrahieren des Texts beschädigt worden. Bitte gib mir die volle Bezeichnung jeder Spalte in der korrekten Reihenfolge, wie sie im Dokument auftaucht. Antworte ausschließlich mit einer Liste im Format [spalte_1, spalte_2, spalte_3]. Wenn du keine Kopfzeile finden kannst, antworte mit [0]. Bericht: 
column names: [0]
Im folgenden erhältst du einen Bericht einer Hauptversammlung. Werden in dem Dokument wiederholt und ausdrücklich Angaben zum auf der Versammlung vertretenen Grundkapital in Prozent gemacht (Also bspw. 'Grundkapital: 30%'? Antworte nur mit [1] oder [0]. Bericht: 
[1]
Im folgenden erhältst du einen Bericht von einer Hauptver

/var/folders/0n/w0w76l850xd5bkqpcpl6v0rr0000gn/T/ipykernel_3236/3976406905.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set_small.at[index, 'Presence_predicted'] = highest_percentage
  1%|          | 1/182 [00:04<12:57,  4.30s/it]

[74.33, 74.13, 13.46, 74.33, 71.35, 71.35, 74.33]
highest_percentage:  74.33
Found file: data/Praesenzen_hv-info/Adler Real Estate AG-1215/ASM/HV-Beschluss zur ordentlichen Hauptversammlung am 28.09.11.pdf
Du bist ein hilfreicher Assistent, der Berichte von Hauptversammlungen auswertet.
Im folgenden erhältst du einen Bericht einer Hauptversammlung. Das Dokument enthält eine Tabelle mit einer Kopfzeile, aber die Kopfzeile ist beim Extrahieren des Texts beschädigt worden. Bitte gib mir die volle Bezeichnung jeder Spalte in der korrekten Reihenfolge, wie sie im Dokument auftaucht. Antworte ausschließlich mit einer Liste im Format [spalte_1, spalte_2, spalte_3]. Wenn du keine Kopfzeile finden kannst, antworte mit [0]. Bericht: 
column names: [0]
Im folgenden erhältst du einen Bericht einer Hauptversammlung. Werden in dem Dokument wiederholt und ausdrücklich Angaben zum auf der Versammlung vertretenen Grundkapital in Prozent gemacht (Also bspw. 'Grundkapital: 30%'? Antworte nur mit [1] oder

  1%|          | 2/182 [00:13<20:05,  6.70s/it]

[73.66, 73.46, 12.60, 73.66, 63.95, 73.69, 66.91, 73.70]
highest_percentage:  73.7
system_prompt:  Du bist ein hilfreicher Assistent, der Berichte von Hauptversammlungen auswertet.
user_prompt:  Im folgenden erhältst du einen Bericht von einer Hauptversammlung. Antworte ausschließlich mit einer Liste im Format [zahl_1, zahl_2, zahl_3], die ausschließlich alle die genannten Prozentzahlen enthält, die sich auf den Prozentsatz des auf der Hauptversammlung vertretenen Grundkapitals beziehen. Wenn du dir nicht absolut sicher bist, antworte mit [0]. Verwende Punkt statt Komma für die Zahlen. Bericht: 


In [9]:
# Log the table to Weights & Biases
wandb.log({"extraction_results": table})

In [12]:
if column_names == "[0]":
    print("yes")

yes


In [20]:
test_set_small

,ID_Key_original,ID_original,Company_original,Year_original,Presence_original,Index_original,Presence_enhanced,Presence_predicted
8,1215.0,105.0,ADLER Real Estate AG (2015),2010.0,NaN,NaN,74.33,NaN
9,1215.0,105.0,ADLER Real Estate AG (2015),2011.0,NaN,NaN,73.70,NaN
10,1215.0,105.0,ADLER Real Estate AG (2015),2012.0,NaN,NaN,70.65,NaN
11,1215.0,105.0,ADLER Real Estate AG (2015),2013.0,NaN,NaN,68.26,NaN
12,1215.0,105.0,ADLER Real Estate AG (2015),2014.0,NaN,NaN,68.39,NaN
...,...,...,...,...,...,...,...,...
1977,14884.0,NaN,Bilfinger SE,2021.0,NaN,NaN,54.04,NaN
1978,14884.0,NaN,Bilfinger SE,2022.0,NaN,NaN,53.71,NaN
1979,14884.0,NaN,Bilfinger SE,2023.0,NaN,NaN,63.14,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.97


## Read contents of PDF

In [21]:
import pdfplumber

# # Now, full_text contains all the text extracted from the PDF
# print(file_path)

file_path = 'data/Praesenzen_hv-info/Amadeus Fire AG-14519/ASM/HV-Beschluss zur ordentlichen Hauptversammlung am 24.05.18.pdf'

with pdfplumber.open(file_path) as pdf:
    # List to store all DataFrames
    dataframes = []
    
    # List to store all texts
    all_texts = []

    # Iterate through each page
    for page_number, page in enumerate(pdf.pages, start=1):
        # Extract text from the page
        page_text = page.extract_text()
        # Store the text
        all_texts.append((page_number, page_text))

        # Extract tables from the page
        tables = page.extract_tables()

        # # Process each table
        # for table_number, table in enumerate(tables, start=1):
        #     # Convert table to DataFrame
        #     df = pd.DataFrame(table[1:], columns=table[0])

        #     # Store the DataFrame for later use
        #     dataframes.append(df)




In [22]:
all_texts[0][1]

'27.5.2018 Hauptversammlung\nAbstimmungsergebnisse der Hauptversammlung der Amadeus FiRe\nAG am 24. Mai 2018 in Frankfurt am Main\nVom Grundkapital in Höhe von EUR 5.198.237, eingeteilt in 5.198.237 auf den Inhaber lautende Stückaktien, waren zur Abstimmung 3.110.435 Stückaktien\nanwesend. Das entspricht einer Präsenz zur Abstimmung von 59,84 Prozent des Grundkapitals.\nJa- Nein-\nAbgegebene Ja- Stimmen Nein- Stimmen\nAnteil am\ngültige Stimmen in % der Stimmen in % der Enthaltungen Ergebnis\nGrundkapital\nStimmen gesamt gültigen gesamt gültigen\nStimmen Stimmen\nTOP 2:\nBeschlussfassung\nBeschluss\nüber die 3.110.345 59,83% 3.110.030 99,99% 315 0,01% 90\nangenommen\nVerwendung des\nBilanzgewinns\nTOP 3:\nBeschlussfassung\nüber die Entlastung Beschluss\n3.091.267 59.47% 3.088.113 99,90% 3.154 0,10% 19.168\nder Mitglieder des angenommen\nVorstands für das\nGeschäftsjahr 2017\nTOP 4:\nBeschlussfassung\nüber die Entlastung Beschluss\n2.530.720 48,68% 2.485.809 98,23% 44.911 1,77% 574.515\